#Downloading and Importing Dependecies


In [ ]:
import networkx as nx
import plotly.graph_objects as go


In [ ]:
!pip install requests
nltk.download('averaged_perceptron_tagger')
nltk.download('semcor')
nltk.download('conll2000')
nltk.download('conll2002')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package semcor to /root/nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


True

In [ ]:
!pip install python-crfsuite


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
from sklearn.model_selection import train_test_split
import requests
import pandas as pd
import json
import nltk
from sklearn.model_selection import train_test_split

In [ ]:
from nltk.tag import CRFTagger

In [ ]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Requesting Data From API

In [ ]:
import requests

# The URL from which the dataset will be downloaded
url = "https://datasets-server.huggingface.co/rows?dataset=arabic_pos_dialect&config=egy&split=train&offset=0&limit=100"

# Send a GET request to the specified URL
response = requests.get(url)

# Check if the response status code is 200 (indicating a successful request)
if response.status_code == 200:
    # If the request was successful, open a file named "arabic_pos_dialect.json" in write binary mode
    with open("arabic_pos_dialect.json", "wb") as f:
        # Write the content of the response (dataset) to the file
        f.write(response.content)
else:
    # If the request was not successful (status code is not 200), print an error message
    print("Error downloading dataset")


In [ ]:
import json

# Open the JSON file in read mode
with open('/content/arabic_pos_dialect.json') as file:
    # Load the content of the JSON file into the 'data' variable
    data = json.load(file)


In [ ]:
# Extract the value associated with the key 'features' from the loaded JSON data and assign it to the variable 'features'. Represents information about the features of the dataset.
features = data['features']


# Extract the value associated with the key 'rows' from the loaded JSON data and assign it to the variable 'rows'. Represents the actual data rows of the dataset.
rows = data['rows']

In [ ]:
extracted_data = {}  # Initialize an empty dictionary named 'extracted_data'


In [ ]:
for feature in features:
    feature_name = feature['name']  # Extract the name of the feature from the current feature dictionary
    extracted_data[feature_name] = []  # Initialize an empty list in the extracted_data dictionary for the current feature

    for row in rows:
        extracted_data[feature_name].append(row['row'][feature_name])  # Extract the value for the current feature from each row and append it to the corresponding list in extracted_data


In [ ]:
#convert to dataframe
df = pd.DataFrame(extracted_data)

In [ ]:
df.head()

,fold,subfold,words,segments,pos_tags
0,4,A,"[ليه, لما, تحب, حد, من, قلبك, يطلع, واطى, ليه,...","[ليه, لما, تحب, حد, من, قلب+ك, يطلع, واطى, ليه...","[PART, PART, V, NOUN, PREP, NOUN+PRON, V, ADJ,..."
1,4,A,"[عارف, بيقولك, ايه, "", إذا, أخطأت, فأحسن, "", ....","[عارف, ب+يقو+ل+ك, ايه, "", إذا, أخطأ+ت, ف+أحسن,...","[ADJ, PROG_PART+V+PREP+PRON, PART, PUNC, PART,..."
2,3,A,"[الحمد, لله, يا, جدعان, الفرسان, اللي, اتمسكوا...","[ال+حمد, ل+الله, يا, جدعان, ال+فرسان, اللي, ات...","[DET+NOUN, PREP+NOUN, PART, NOUN, DET+NOUN, PA..."
3,1,A,"[بحس, بشخصيتي, القوية, لما, اقول, لاخويا, اعمل...","[ب+حس, ب+شخصي+ت+ي, ال+قوي+ة, لما, اقول, ل+اخو+...","[PROG_PART+V, PREP+NOUN+NSUFF+PRON, DET+ADJ+NS..."
4,5,A,"[@ahmedabodsheesh, يا, باشا, دي, مش, محتاجه, د...","[@ahmedabodsheesh, يا, باشا, دي, مش, محتاج+ه, ...","[MENTION, PART, NOUN, PRON, PART, ADJ+NSUFF, N..."


In [ ]:
df.shape

(100, 5)

# CRFTagger


In [ ]:
# Convert the 'word' and 'pos' columns to lists
words_list = df['words'].tolist()
pos_tags_list = df['pos_tags'].tolist()

# Create a list of tuples containing the words and POS tags for each row
data = list(zip(words_list, pos_tags_list))

In [ ]:
# Flatten the train_data and test_data lists
train_data_flat = [tuple(zip(sentence[0], sentence[1])) for sentence in data]

# Train the CRF Tagger
crf_tagger = CRFTagger()
crf_tagger.train(train_data_flat, 'model.crf.tagger')

In [ ]:
# Preprocess new, unseen data and tokenize it
new_data = "لعب الولد الكرة في المدرسة الكبيرة"
tokens = nltk.word_tokenize(new_data)

# Use the trained CRF Tagger to predict POS tags for the new data
predicted_tags = crf_tagger.tag(tokens)

# Print the predicted POS tags
for token, tag in predicted_tags:
    print(token, tag)

لعب ADJ
الولد NOUN
الكرة NOUN+NSUFF
في PREP
المدرسة NOUN+NSUFF
الكبيرة ADJ+NSUFF


In [ ]:
# Preprocess new, unseen data and tokenize it
new_data = "أرحب بيكم"
tokens = nltk.word_tokenize(new_data)

# Use the trained CRF Tagger to predict POS tags for the new data
predicted_tags = crf_tagger.tag(tokens)

# Print the predicted POS tags
for token, tag in predicted_tags:
    print(token, tag)

أرحب V
بيكم PREP+PRON


In [ ]:
# Preprocess new, unseen data and tokenize it
new_data = " أدخلوا مصر ان شاء الله امنين "
tokens = nltk.word_tokenize(new_data)

# Use the trained CRF Tagger to predict POS tags for the new data
predicted_tags = crf_tagger.tag(tokens)

# Print the predicted POS tags
for token, tag in predicted_tags:
    print(token, tag)

أدخلوا V+PRON
مصر NOUN
ان PART
شاء NOUN
الله NOUN+NSUFF
امنين ADJ+NSUFF


# NetworkX and Plotly(Interactive plot)

In [ ]:
# Create an empty graph
graph = nx.Graph()

# Add nodes
for token, tag in predicted_tags:
    graph.add_node(token, pos_tag=tag)

# Add edges
for i in range(len(predicted_tags) - 1):
    graph.add_edge(predicted_tags[i][0], predicted_tags[i+1][0])


In [ ]:
# Choose a layout for the graph (e.g., spring_layout)
layout = nx.spring_layout(graph)


In [ ]:
# Create a Plotly figure
fig = go.Figure()

# Add nodes to the figure
for node in graph.nodes:
    x, y = layout[node]
    tag = graph.nodes[node]['pos_tag']  # Get the POS tag for the node
    fig.add_trace(go.Scatter(x=[x], y=[y], mode='markers', name=node, text=node+' ('+tag+')',
                             marker=dict(size=30)))  # Increase the size of the markers

# Add edges to the figure
for edge in graph.edges:
    x0, y0 = layout[edge[0]]
    x1, y1 = layout[edge[1]]
    fig.add_trace(go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines'))

# Set the title and layout of the figure
fig.update_layout(title='POS Tag Network',
                  showlegend=False,
                  hovermode='closest')

fig.show()
